In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10963,2024-04-09,Israel Superliga,13:00,Maccabi Tel Aviv,Nes Ziona,1.10,6.48,171.5,1.87,1.87,0.0,0.00,0.00,jkVBPPhA,0.909091,0.154321,0.534759,0.534759,0.063412,111.668,6.427338,0.057558,3.0,0.000000,0.000000,104.86,1.174,0.103586,0.088233,97.0,145.738,47.089986,0.323114,1.2,1.643168,1.369306,146.02,1.896,0.692662,0.365328,-7.0,98,98,1.07,1.49,105.314,186.592,1.003756,0.000000,NaN,0.68,0.136,0.735294,0.000000,0.0,0.000000,-3.25,-0.650,-8.430769,0.519873,0.4,-0.119873
10964,2024-04-09,Israel Liga Leumit,12:30,Hapoel Bnei Kfar Kasem,Maccabi Maale Adumim,1.37,2.91,158.5,1.85,1.85,0.0,0.00,0.00,nc68npw6,0.729927,0.343643,0.540541,0.540541,0.073570,218.852,67.786066,0.309735,1.8,1.643168,0.912871,321.36,2.698,0.639547,0.237045,-13.0,198.320,69.092009,0.348386,1.2,1.643168,1.369306,150.41,2.602,0.856720,0.329255,-12.0,103,89,3.12,1.69,152.406,241.270,0.508853,0.000000,NaN,0.17,0.034,10.882353,0.000000,0.0,0.000000,3.71,0.742,2.574124,0.000000,0.0,0.000000
10965,2024-04-09,Israel Liga Leumit,13:00,Maccabi Haifa,Maccabi Rishon,2.19,1.63,168.5,1.81,1.89,0.0,0.00,0.00,IHEijSws,0.456621,0.613497,0.552486,0.529101,0.070118,207.930,74.361699,0.357629,1.2,1.643168,1.369306,115.36,2.550,0.594685,0.233210,-9.0,170.352,37.634050,0.220919,1.8,1.643168,0.912871,168.48,1.930,0.622254,0.322411,44.0,56,104,2.06,1.62,211.974,208.528,0.207319,0.030578,NaN,5.03,1.006,1.182903,0.000000,0.0,0.000000,-3.26,-0.652,-0.966258,0.000000,0.0,0.000000
10966,2024-04-09,Israel Liga Leumit,13:30,Ironi Nahariya,Ramat Hasharon,1.27,3.53,163.5,1.83,1.87,0.0,0.00,0.00,Eq80l68g,0.787402,0.283286,0.546448,0.534759,0.070688,126.078,19.994260,0.158586,2.4,1.341641,0.559017,125.02,1.412,0.134981,0.095596,34.0,163.128,56.197640,0.344500,0.6,1.341641,2.236068,162.40,2.308,0.787763,0.341318,-41.0,94,70,1.33,2.32,126.078,215.740,0.665859,0.015289,NaN,0.73,0.146,1.849315,0.672079,0.6,-0.072079,-5.00,-1.000,-2.530000,0.397239,0.1,-0.297239
10967,2024-04-09,Israel Liga Leumit,13:30,Gilboa Galil,Maccabi Rehovot,2.19,1.63,158.5,1.87,1.79,0.0,0.00,0.00,hz74mQNa,0.456621,0.613497,0.534759,0.558659,0.070118,152.504,48.500181,0.318026,1.8,1.643168,0.912871,161.19,2.020,0.609508,0.301737,8.0,119.778,15.683986,0.130942,1.2,1.643168,1.369306,111.72,1.452,0.208375,0.143509,-20.0,81,84,1.99,1.33,116.848,145.362,0.207319,0.030912,NaN,-1.34,-0.268,-4.440299,0.000000,0.0,0.000000,0.19,0.038,16.578947,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11039,2024-04-09,Bósnia E Herzegovina Prvenstvo Bih Feminina,15:00,Lavovi F,Jedinstvo Dzenex F,1.83,1.83,134.5,1.83,1.83,0.0,0.00,0.00,nLW4cUh0,0.546448,0.546448,0.546448,0.546448,0.092896,633.896,617.062850,0.973445,2.4,1.341641,0.559017,1350.50,8.858,8.786218,0.991896,87.0,224.014,135.257573,0.603791,1.2,1.643168,1.369306,172.80,4.944,4.533335,0.916937,-72.0,73,54,18.50,3.20,0.000,373.346,0.000000,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-0.65,-0.130,-6.384615,0.000000,0.0,0.000000
11040,2024-04-09,Austrália Nbl1 Sul,05:30,Diamond Valley F,Keilor Thunder F,3.69,1.25,152.5,1.85,1.85,0.0,0.00,0.00,f73qJ1ji,0.2710

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10965,13:00,Israel Liga Leumit,Maccabi Haifa,Maccabi Rishon,2.19,Back Home
11000,00:15,México Cibacopa,Rayos de Hermosillo,Astros,2.15,Back Home
11005,23:00,Eua Nba,Phoenix Suns,Los Angeles Clippers,1.63,Back Home
11020,13:00,Rússia Superliga,MBA Moscow 2,Tambov,2.26,Back Home
